# ML Cup Event #3

In [1]:
import pandas as pd
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.24.1.


In [3]:
from pathlib import Path
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline 

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [4]:
import os
os.getcwd()

'C:\\Users\\talal\\Downloads'

In [5]:
train = pd.read_csv("C:/Users/talal/OneDrive/Documents/Queens/MMA 869 - Machine Learning/Project/training_set_features.csv", index_col="respondent_id")
labels = pd.read_csv("C:/Users/talal/OneDrive/Documents/Queens/MMA 869 - Machine Learning/Project/training_set_labels.csv", index_col="respondent_id")
test = pd.read_csv("C:/Users/talal/OneDrive/Documents/Queens/MMA 869 - Machine Learning/Project/test_set_features.csv", index_col="respondent_id")
train.info()
train.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26707 entries, 0 to 26706
Data columns (total 35 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   h1n1_concern                 26615 non-null  float64
 1   h1n1_knowledge               26591 non-null  float64
 2   behavioral_antiviral_meds    26636 non-null  float64
 3   behavioral_avoidance         26499 non-null  float64
 4   behavioral_face_mask         26688 non-null  float64
 5   behavioral_wash_hands        26665 non-null  float64
 6   behavioral_large_gatherings  26620 non-null  float64
 7   behavioral_outside_home      26625 non-null  float64
 8   behavioral_touch_face        26579 non-null  float64
 9   doctor_recc_h1n1             24547 non-null  float64
 10  doctor_recc_seasonal         24547 non-null  float64
 11  chronic_med_condition        25736 non-null  float64
 12  child_under_6_months         25887 non-null  float64
 13  health_worker   

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
respondent_id,,,,,,,,,,,,,,,,,,,,,
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN
1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe
2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo
3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,Below Poverty,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN
4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,...,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb


In [6]:
labels_h1n1 = labels[['h1n1_vaccine']]
labels_seasonal = labels[['seasonal_vaccine']]

In [7]:
train.drop(['health_insurance','hhs_geo_region','employment_industry','employment_occupation'], axis = 1, inplace = True)
test.drop([ 'health_insurance','hhs_geo_region','employment_industry','employment_occupation'], axis = 1, inplace = True)

In [8]:
#First of all let's check how many absent values do we have:
null_value_stats = train.isnull().sum(axis=0)
null_value_stats[null_value_stats != 0]

h1n1_concern                     92
h1n1_knowledge                  116
behavioral_antiviral_meds        71
behavioral_avoidance            208
behavioral_face_mask             19
behavioral_wash_hands            42
behavioral_large_gatherings      87
behavioral_outside_home          82
behavioral_touch_face           128
doctor_recc_h1n1               2160
doctor_recc_seasonal           2160
chronic_med_condition           971
child_under_6_months            820
health_worker                   804
opinion_h1n1_vacc_effective     391
opinion_h1n1_risk               388
opinion_h1n1_sick_from_vacc     395
opinion_seas_vacc_effective     462
opinion_seas_risk               514
opinion_seas_sick_from_vacc     537
education                      1407
income_poverty                 4423
marital_status                 1408
rent_or_own                    2042
employment_status              1463
household_adults                249
household_children              249
dtype: int64

In [9]:
numeric_cols = train.columns[train.dtypes != 'object'].values
non_numeric_cols = train.columns[train.dtypes == 'object'].values

In [10]:
numeric_cols

array(['h1n1_concern', 'h1n1_knowledge', 'behavioral_antiviral_meds',
       'behavioral_avoidance', 'behavioral_face_mask',
       'behavioral_wash_hands', 'behavioral_large_gatherings',
       'behavioral_outside_home', 'behavioral_touch_face',
       'doctor_recc_h1n1', 'doctor_recc_seasonal',
       'chronic_med_condition', 'child_under_6_months', 'health_worker',
       'opinion_h1n1_vacc_effective', 'opinion_h1n1_risk',
       'opinion_h1n1_sick_from_vacc', 'opinion_seas_vacc_effective',
       'opinion_seas_risk', 'opinion_seas_sick_from_vacc',
       'household_adults', 'household_children'], dtype=object)

In [11]:
non_numeric_cols

array(['age_group', 'education', 'race', 'sex', 'income_poverty',
       'marital_status', 'rent_or_own', 'employment_status', 'census_msa'],
      dtype=object)

In [12]:
train['rent_or_own'] = np.where(train['rent_or_own']=="Own", 1, 0)
test['rent_or_own'] = np.where(test['rent_or_own']=="Own", 1, 0)

In [13]:
train['employment_status'] = np.where(train['employment_status']=="Employed",1,0)
test['employment_status'] = np.where(test['employment_status']=="Employed",1,0)

In [14]:
train['marital_status'] =np.where(train['marital_status']=="Married",1,0)
test['marital_status'] =np.where(test['marital_status']=="Married",1,0)

In [15]:
train['census_msa'] = np.where(train['census_msa']=='MSA, Principle City',1,0)
test['census_msa'] = np.where(test['census_msa']=='MSA, Principle City',1,0)

In [16]:
scale_mapper_income = {'Below Poverty':1, '<= $75,000, Above Poverty':2, '> $75,000':3}
train['income_poverty'] = train['income_poverty'].replace(scale_mapper_income)
test['income_poverty'] = test['income_poverty'].replace(scale_mapper_income)

In [17]:
scale_mapper_age = {'18 - 34 Years':1, '35 - 44 Years':2, '45 - 54 Years':3, '55 - 64 Years':4,'65+ Years':5}
train['age_group'] = train['age_group'].replace(scale_mapper_age)
test['age_group'] = test['age_group'].replace(scale_mapper_age)

In [18]:
scale_mapper_education = {'< 12 Years':1, '12 Years':2, 'Some College':3, 'College Graduate':4}
train['education'] = train['education'].replace(scale_mapper_education)
test['education'] = test['education'].replace(scale_mapper_education)

In [19]:
non_numeric_cols = train.columns[train.dtypes == 'object'].values
non_numeric_cols
numeric_cols = train.columns[train.dtypes != 'object'].values

array(['race', 'sex'], dtype=object)

In [20]:
((train.isnull() | train.isna()).sum() * 100 / train.index.size).round(2)

h1n1_concern                    0.34
h1n1_knowledge                  0.43
behavioral_antiviral_meds       0.27
behavioral_avoidance            0.78
behavioral_face_mask            0.07
behavioral_wash_hands           0.16
behavioral_large_gatherings     0.33
behavioral_outside_home         0.31
behavioral_touch_face           0.48
doctor_recc_h1n1                8.09
doctor_recc_seasonal            8.09
chronic_med_condition           3.64
child_under_6_months            3.07
health_worker                   3.01
opinion_h1n1_vacc_effective     1.46
opinion_h1n1_risk               1.45
opinion_h1n1_sick_from_vacc     1.48
opinion_seas_vacc_effective     1.73
opinion_seas_risk               1.92
opinion_seas_sick_from_vacc     2.01
age_group                       0.00
education                       5.27
race                            0.00
sex                             0.00
income_poverty                 16.56
marital_status                  0.00
rent_or_own                     0.00
e

In [21]:
numeric_preprocessing_steps = Pipeline([
    ('standard_scaler', StandardScaler()),
    ('simple_imputer', SimpleImputer(strategy = 'median'))
])

non_numeric_preprocessing_steps = Pipeline([
     ('simple_imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('one_hot_encoder', OneHotEncoder())
])

preprocessor = ColumnTransformer(
    transformers = [
        ('numeric', numeric_preprocessing_steps, numeric_cols),
        ('non_numeric', non_numeric_preprocessing_steps, non_numeric_cols)  
    ],
    remainder = 'drop'
)

In [22]:
X_train, X_eval, y_train, y_eval = train_test_split(
    train,
    labels_h1n1,
    test_size=0.2,
    shuffle=True,
    stratify=labels_h1n1,
    random_state=100
)

In [23]:
X_train_preprocess = pd.DataFrame(preprocessor.fit_transform(X_train))
X_eval_preprocess =  pd.DataFrame(preprocessor.transform(X_eval))


print ('X_train_preprocess.shape' , X_train_preprocess.shape)
print ('X_eval_preprocess.shape' , X_eval_preprocess.shape)

X_train_preprocess.shape (21365, 35)
X_eval_preprocess.shape (5342, 35)


In [24]:
X_train_preprocess.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21365 entries, 0 to 21364
Data columns (total 35 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       21365 non-null  float64
 1   1       21365 non-null  float64
 2   2       21365 non-null  float64
 3   3       21365 non-null  float64
 4   4       21365 non-null  float64
 5   5       21365 non-null  float64
 6   6       21365 non-null  float64
 7   7       21365 non-null  float64
 8   8       21365 non-null  float64
 9   9       21365 non-null  float64
 10  10      21365 non-null  float64
 11  11      21365 non-null  float64
 12  12      21365 non-null  float64
 13  13      21365 non-null  float64
 14  14      21365 non-null  float64
 15  15      21365 non-null  float64
 16  16      21365 non-null  float64
 17  17      21365 non-null  float64
 18  18      21365 non-null  float64
 19  19      21365 non-null  float64
 20  20      21365 non-null  float64
 21  21      21365 non-null  float64
 22

In [25]:
train_df_preprocess = pd.DataFrame(preprocessor.transform(train))
test_df_preprocess = pd.DataFrame(preprocessor.transform(test))
print ('train_df_preprocess.shape' , train_df_preprocess.shape)
print ('test_df_preprocess.shape' , test_df_preprocess.shape)

train_df_preprocess.shape (26707, 35)
test_df_preprocess.shape (26708, 35)


## Feature Selection

In [98]:
 ## Using to pick certain features that have the best relationship to the performance variable.

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [99]:
x_seas = train.iloc[:,:-2] ## Independent variables
y_seas= train.iloc[:,-1]  ## Target variable 

In [100]:
## Apply select K best to extract best features 
bestfeatures = SelectKBest(chi2,k=5)
fit = bestfeatures.fit(x_seas,y_seas)

In [101]:
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(x_seas.columns)

In [102]:
## Concatenating two dataframes for better visualisations 
featurescores = pd.concat([dfcolumns,dfscores],axis=1)
featurescores.columns = ['feature_names','Scores']

In [103]:
featurescores

,feature_names,Scores
0,Id,104.377610
1,num_times_pregnant,72.127562
2,plasma_glucose,1101.543699
3,DBP,22.315142
4,triceps_skin,32.719501
5,serum_insulin,1542.993522
6,BMI,103.726397
7,pedigree,3.040570


In [51]:
## printing top features 
print(featurescores.nlargest(22,'Scores'))

        feature_names       Scores
5       serum_insulin  1542.993522
2      plasma_glucose  1101.543699
0                  Id   104.377610
6                 BMI   103.726397
1  num_times_pregnant    72.127562
4        triceps_skin    32.719501
3                 DBP    22.315142
7            pedigree     3.040570


In [104]:
## Defining the dependent and independent (target) variables

x=train.iloc[:,:-2]
y=train.iloc[:,-2]

In [105]:
y

0      44
1      23
2      46
3      25
4      21
       ..
571    26
572    27
573    38
574    29
575    21
Name: age, Length: 576, dtype: int64

In [8]:
X.info()
X.shape
X.head()

X_train.info()
X_train.shape
X_train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 576 entries, 0 to 575
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   num_times_pregnant  576 non-null    int64  
 1   plasma_glucose      576 non-null    int64  
 2   DBP                 576 non-null    int64  
 3   triceps_skin        576 non-null    int64  
 4   serum_insulin       576 non-null    int64  
 5   BMI                 576 non-null    float64
 6   pedigree            576 non-null    float64
 7   age                 576 non-null    int64  
dtypes: float64(2), int64(6)
memory usage: 36.1 KB


(576, 8)

,num_times_pregnant,plasma_glucose,DBP,triceps_skin,serum_insulin,BMI,pedigree,age
0,13,129,0,30,0,39.9,0.569,44
1,4,129,86,20,270,35.1,0.231,23
2,3,61,82,28,0,34.4,0.243,46
3,2,81,72,15,76,30.1,0.547,25
4,0,102,75,23,0,0.0,0.572,21


<class 'pandas.core.frame.DataFrame'>
Int64Index: 460 entries, 437 to 102
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   num_times_pregnant  460 non-null    int64  
 1   plasma_glucose      460 non-null    int64  
 2   DBP                 460 non-null    int64  
 3   triceps_skin        460 non-null    int64  
 4   serum_insulin       460 non-null    int64  
 5   BMI                 460 non-null    float64
 6   pedigree            460 non-null    float64
 7   age                 460 non-null    int64  
dtypes: float64(2), int64(6)
memory usage: 32.3 KB


(460, 8)

,num_times_pregnant,plasma_glucose,DBP,triceps_skin,serum_insulin,BMI,pedigree,age
437,6,102,82,0,0,30.8,0.180,36
63,7,142,60,33,190,28.8,0.687,61
208,1,119,44,47,63,35.5,0.280,25
60,8,143,66,0,0,34.9,0.129,41
15,0,123,88,37,0,35.2,0.197,29


In [9]:
np.bincount(y)

array([377, 199], dtype=int64)

In [57]:
#checking null values
null_value_stats = train.isnull().sum(axis=0)
null_value_stats[null_value_stats != 0]

Series([], dtype: int64)

# Build Model

In [26]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()  ## model creation and fitting
rf.fit(X_train_preprocess,y_train)
y_hat=rf.predict(X_eval_preprocess)

<ipython-input-26-c17f7b87d182>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train_preprocess,y_train)


RandomForestClassifier()

In [27]:
## evaluating model created 
from sklearn.metrics import accuracy_score,classification_report,log_loss,roc_auc_score,confusion_matrix
accuracy_score(y_eval,y_hat)

0.8330213403219768

In [28]:
print(confusion_matrix(y_eval,y_hat))
print(log_loss(y_eval,y_hat))
print(classification_report(y_eval,y_hat))

[[3996  211]
 [ 681  454]]
5.767270172092494
              precision    recall  f1-score   support

           0       0.85      0.95      0.90      4207
           1       0.68      0.40      0.50      1135

    accuracy                           0.83      5342
   macro avg       0.77      0.67      0.70      5342
weighted avg       0.82      0.83      0.82      5342



In [29]:
y_hat = rf.predict_proba(X_eval_preprocess)[:,1]

In [30]:
roc_auc_score(y_eval,y_hat)

0.8184474376144646

### Decision Tree (Extra Trees Classifier)

In [26]:
##Extra Trees Decision Tree
from sklearn.ensemble import ExtraTreesClassifier
Ex=ExtraTreesClassifier()  ## model creation 
Ex.fit(X_train_preprocess,y_train)
y_hat_Ex=Ex.predict(X_eval_preprocess)

<ipython-input-26-7031d1fbd1ae>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  Ex.fit(X_train_preprocess,y_train)


ExtraTreesClassifier()

In [27]:
from sklearn.metrics import accuracy_score,classification_report
accuracy_score(y_eval,y_hat_Ex)

0.832459752901535

In [28]:
print(classification_report(y_eval,y_hat_Ex))

              precision    recall  f1-score   support

           0       0.86      0.95      0.90      4207
           1       0.67      0.41      0.51      1135

    accuracy                           0.83      5342
   macro avg       0.76      0.68      0.70      5342
weighted avg       0.82      0.83      0.82      5342



In [30]:
print("Train Accuracy:",Ex.score(X_train_preprocess, y_train))
print("Test Accuracy:", Ex.score(X_eval_preprocess, y_eval))

Train Accuracy: 0.9999063889538966
Test Accuracy: 0.832459752901535


### Gradient Boosting

In [26]:
#Gradient Boosting

from sklearn.ensemble import GradientBoostingClassifier

gbc = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
                                 max_depth=1, random_state=0).fit(X_train_preprocess, y_train)

gbc_y_pred = gbc.predict(X_eval_preprocess)


C:\Users\talal\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [27]:
from sklearn.metrics import accuracy_score,classification_report
accuracy_score(y_eval,gbc_y_pred)

0.8358292774241857

In [29]:
print(classification_report(y_eval,gbc_y_pred))

              precision    recall  f1-score   support

           0       0.86      0.95      0.90      4207
           1       0.68      0.42      0.52      1135

    accuracy                           0.84      5342
   macro avg       0.77      0.69      0.71      5342
weighted avg       0.82      0.84      0.82      5342



## Decision Tree

In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score

In [28]:
#Decision Tree



dt = DecisionTreeClassifier(max_depth=10, random_state=45)
dtfit = dt.fit(X_train_preprocess, y_train)

dt_y_pred = dt.predict(X_eval_preprocess)

In [29]:
from sklearn.metrics import accuracy_score,classification_report
accuracy_score(y_eval,dt_y_pred)

0.8214152002995133

In [30]:
#instead of just doing the tree, trying to find the right parameters. 

param_dist = {"max_depth": [2,20,60,70,100],
              "max_features": [1,2,3,4,5,6,7,8,9],
              "min_samples_leaf": [1,20,40],
              "criterion": ["gini", "entropy"]}

tree = DecisionTreeClassifier()

# RandomizedSearchCV object: tree_cv
tree_cv = RandomizedSearchCV(tree, param_distributions=param_dist, cv=10, n_iter=100, random_state=40)

# Fit it to the data
tree_cv.fit(X_train_preprocess, y_train)

# Print the tuned parameters and score
print("Tuned Decision Tree Parameters: {}".format(tree_cv.best_params_))

RandomizedSearchCV(cv=10, estimator=DecisionTreeClassifier(), n_iter=100,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [2, 20, 60, 70, 100],
                                        'max_features': [1, 2, 3, 4, 5, 6, 7, 8,
                                                         9],
                                        'min_samples_leaf': [1, 20, 40]},
                   random_state=40)

Tuned Decision Tree Parameters: {'min_samples_leaf': 20, 'max_features': 7, 'max_depth': 100, 'criterion': 'gini'}


In [31]:
#Using the tuned decision tree parameters

dt2 = DecisionTreeClassifier(criterion= 'gini', max_depth= 100, 
                                     max_features= 7, min_samples_leaf= 20, 
                                     random_state=40)
dt2fit = dt2.fit(X_train_preprocess, y_train)
dt_y_pred2 = dt2.predict(X_eval_preprocess)

In [32]:
from sklearn.metrics import accuracy_score,classification_report
accuracy_score(y_eval,dt_y_pred2)

0.8212280044926994

## LGBM

In [10]:
param1 = {'learning_rate': 0.05,
               'max_depth': 12,
               'min_child_weight':1.75,
               'colsample_bytree':1,
               'subsample': 1, 'num_leaves' : 200,'random_state': 1450,
               'eval_metric': "auc",
               'objective': "binary"}


In [11]:
#Using LGBM

from lightgbm import LGBMClassifier 

model=LGBMClassifier(**param1)


In [12]:
model.fit(X_train, y_train, early_stopping_rounds=1000, eval_set = [(X_val, y_val)], eval_metric = 'auc', verbose = 1000)

[LightGBM] [Warning] Unknown parameter: eval_metric


C:\Users\talal\Anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\talal\Anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


LGBMClassifier(colsample_bytree=1, eval_metric='auc', learning_rate=0.05,
               max_depth=12, min_child_weight=1.75, num_leaves=200,
               objective='binary', random_state=1450, subsample=1)

# Estimate Model Performance

In [30]:
from sklearn.metrics import confusion_matrix

pred_h1n1 = gbc.predict_proba(X_eval_preprocess)
pred_h1n1

confusion_matrix(y_eval, pred_h1n1)

array([[0.94312068, 0.05687932],
       [0.99474214, 0.00525786],
       [0.95334942, 0.04665058],
       ...,
       [0.73566715, 0.26433285],
       [0.93128087, 0.06871913],
       [0.97808888, 0.02191112]])

ValueError: Classification metrics can't handle a mix of binary and continuous-multioutput targets

In [31]:
#seasonal prediction
pred_seasonal = gbc.predict_proba(X_eval_preprocess)
pred_seasonal

confusion_matrix(y_eval, pred_seasonal)

array([[0.94312068, 0.05687932],
       [0.99474214, 0.00525786],
       [0.95334942, 0.04665058],
       ...,
       [0.73566715, 0.26433285],
       [0.93128087, 0.06871913],
       [0.97808888, 0.02191112]])

ValueError: Classification metrics can't handle a mix of binary and continuous-multioutput targets

In [32]:
#seasonal
y_pred_seasonal = pd.DataFrame(
    {
        "seasonal_vaccine": pred_seasonal[:, 1],
        
    },
    index = y_eval.index
)
print("y_pred_seasonal.shape:", y_pred_seasonal.shape)

y_pred_seasonal.shape: (5342, 1)


In [33]:
#H1N1
y_pred_h1n1 = pd.DataFrame(
    {
        "h1n1_vaccine": pred_h1n1[:, 1],
        
    },
    index = y_eval.index
)
print("y_pred_h1n1.shape:", y_pred_h1n1.shape)

y_pred_h1n1.shape: (5342, 1)


In [34]:
roc_auc_score(y_eval, y_pred_h1n1)

0.8263810368496392

In [35]:
roc_auc_score(y_eval, y_pred_seasonal)

0.8263810368496392

In [36]:
gbc.fit(train_df_preprocess, labels_h1n1)

C:\Users\talal\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


GradientBoostingClassifier(learning_rate=1.0, max_depth=1, random_state=0)

In [37]:
gbc.fit(train_df_preprocess, labels_seasonal)

C:\Users\talal\Anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


GradientBoostingClassifier(learning_rate=1.0, max_depth=1, random_state=0)

In [39]:
preds1_h1n1 = gbc.predict_proba(test_df_preprocess)

In [40]:
preds1_seasonal = gbc.predict_proba(test_df_preprocess)

In [41]:
y_preds1_h1n1 = pd.DataFrame(
    {
        "h1n1_vaccine": preds1_h1n1[:, 1],
        
    },
    index = test.index
)
print("y_preds1_h1n1.shape:", y_preds1_h1n1.shape)

y_preds1_h1n1.shape: (26708, 1)


In [42]:
y_preds1_seasonal = pd.DataFrame(
    {
        "seasonal_vaccine": preds1_seasonal[:, 1],
        
    },
    index = test.index
)
print("y_preds1_seasonal.shape:", y_preds1_seasonal.shape)

y_preds1_seasonal.shape: (26708, 1)


## Create Submission File

In [46]:
y_preds1_h1n1.to_csv('model_h1n1.csv', index=True)

In [47]:
h1n1_df = pd.read_csv("C:/Users/talal/Downloads/model_h1n1.csv", 
                            index_col="respondent_id")

In [48]:
joined_df = h1n1_df.join(y_preds1_seasonal)
joined_df.head()

,h1n1_vaccine,seasonal_vaccine
respondent_id,,
26707,0.287236,0.287236
26708,0.040046,0.040046
26709,0.778748,0.778748
26710,0.851667,0.851667
26711,0.382043,0.382043


In [49]:
joined_df.to_csv('gbc.csv', index=True)